In [ ]:
#建立攝影機
import requests

media_server_ip = input("請輸入Ant Media Server IP (e.g. 61.222.163.86):")
media_server_port = int(input("請輸入 Port (預設 5080):") or 5080)
username = input("請輸入username (e.g. admin)")
password = input("請輸入password(e.g. ez008313)")
zone = input("請輸入機台區域 (e.g. A/B/C/D):").strip()
subnet = int(input("請輸入網段(e.g. 11/12/13/14):").strip())
start_id = int(input("請輸入起始 ID (包含):"))
end_id = int(input("請輸入結束 ID (包含):"))

#ip變數規則
def ip_location(subnet: str, id: int) -> int:
    if subnet == "11":
        return id
    elif subnet == "12":
        return id - 200
    elif subnet in ("14", "15"):
        return id - 300
    elif subnet == "13":
        if id > 400:
            return id - 400
        elif id > 300:
            return id - 300
        else:
            return id
    else:
        raise ValueError(f"Unknown subnet: {subnet}")

results = []
fail_count = 0

for id in range(start_id, end_id + 1):
    suffix = ip_location(subnet, id)
    ip_addr = f"192.168.{subnet}.{suffix}"
    name = f"{zone}{id if id > 100 else f'0{id}'}"

    url = f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts/create?autoStart=true"
    data = {
        "hlsViewerCount": 0,
        "dashViewerCount": 0,
        "webRTCViewerCount": 0,
        "rtmpViewerCount": 0,
        "mp4Enabled": 0,
        "playlistLoopEnabled": True,
        "autoStartStopEnabled": False,
        "plannedStartDate": 0,
        "playListItemList": [],
        "ipAddr": ip_addr,
        "name": name,
        "username": username,
        "password": password,
        "type": "ipCamera",
    }

    response = requests.post(url, json=data, timeout= 10)

    if response.status_code == 200:
        results.append({'id': id, 'name': name})
    else:
        fail_count += 1
        print(f"❌ {name} 建立失敗:HTTP {response.status_code} → {response.text}")

success_count = len(results)
total_count = end_id - start_id +1
print(f"✅ 建立完成: 共{total_count}，成功 {success_count} 筆，失敗{fail_count}筆。")


In [ ]:
#匯出streamID
import requests
import csv

media_server_ip = input("請輸入Ant Media Server IP (e.g. 61.222.163.86)")
media_server_port = int(input("請輸入 Port (預設 5080):") or 5080)

url = f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts/list/0/1000?sort_by=name&order_by=asc"

r = requests.get(url, timeout= 10)
r.raise_for_status()
data = r.json()

# 排序依據是 name 裡的數字部分（例如 'E123' → 123）
data.sort(key=lambda x: int(x["name"][1:]))

# 輸出 SQL 指令
with open("stream_id_sql.csv", mode="w", newline="", encoding="utf-8") as file_sql:
    writer_sql = csv.writer(file_sql)
    writer_sql.writerow(["ID", "Name", "Stream ID", "SQL Command"])
    for i in data:
        name = i["name"]
        id = int(name[1:])
        stream_id = i["streamId"]
        sql = f'UPDATE `machines` SET `stream_name` = "{stream_id}" WHERE `id` = {id};'
        writer_sql.writerow([id, name, stream_id, sql])

# 輸出 填表用 StreamID
with open("stream_id_sheet.csv", mode="w" , newline="", encoding="utf-8")as file_sheet:
    writer_list = csv.writer(file_sheet, quoting=csv.QUOTE_NONE, escapechar="\\")
    writer_list.writerow(["Name", "Stream ID"]) 
    for i in data:
        writer_list.writerow([i["name"], i["streamId"]])
        
print("✅ 匯出成功")


In [ ]:
# 刪除區域攝影機
import requests

media_server_ip = input("請輸入Ant Media Server IP (e.g. 61.222.163.86):")
media_server_port = int(input("請輸入 Port (預設 5080):") or 5080)

ip = f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts"
camera_list = f"{ip}/list/0/1000?sort_by=name&order_by=asc"

resp = requests.get(camera_list, timeout= 10)
resp.raise_for_status()
data = resp.json()

zone = input("請輸入所要刪除的機台區域 (e.g. A/B/C/D):").strip()

delete = [i["streamId"] for i in data if i["name"].startswith(zone)]

if not delete:
    print(f"⚠️ 沒有找到任何以「{zone}」開頭的機台。")
else:
    print(delete)

    for id in delete:
        url = f"{ip}/{id}"
        r = requests.delete(url, timeout= 5)
        if r.status_code in (200, 204):
            print("刪除成功")
        else:
            print(f"❌ 刪除失敗：{id} → HTTP {r.status_code},{r.text}")
    
    print(f"\n🔚 完成，共刪除 {len(delete)} 支「{zone}」開頭的機台。")